In [ ]:
import sys, os, glob
import numpy, scipy

path = "../work/run-scf-kpt"
assert os.path.exists(path)

# find all the out.log files
out_logs = glob.glob(os.path.join(path, "**", "out.log"), recursive=True)

def get_info_from_out_log(out_log):
    d = {}
    kmesh = out_log.split("/")[4]
    kmesh = [int(x) for x in kmesh.split("-")]
    d["kmesh"] = numpy.prod(kmesh)
    d["method"] = "-".join(out_log.split("/")[5].split("-")[:-1])
    d["ncpu"] = int(out_log.split("/")[5].split("-")[-1])
    d["cell"] = out_log.split("/")[3]
    d["natm"] = 2 if d["cell"] == "diamond-prim" else 8

    with open(out_log, "r") as f:
        table = []
        for line in f:
            # if start with CPU time for * sec, wall time * sec
            print(line)
            if "CPU time" in line:
                key_word = line.split()[3]
                wall_time = line.split()[8]
                d["t_" + key_word] = float(wall_time)

            if "###" in line:
                table.append(line)

        assert len(table) == 2
        keys = table[0][3:].split(",")
        values = table[1][3:].split(",")
        for key, value in zip(keys, values):
            d[key.strip()] = float(value.strip())

    return d

data = []
# read the out.log files
for out_log in out_logs:
    print(out_log)
    d = get_info_from_out_log(out_log)
    data.append(d)

../work/run-scf-kpt/diamond-prim/2-4-4/fftisdf-jy-1/100/c0-20.0-k0-80.0/out.log
    CPU time for build     73.62 sec, wall time     73.95 sec

    CPU time for vj      1.34 sec, wall time      1.35 sec

    CPU time for vk      0.58 sec, wall time      0.58 sec

###  ke_cutoff,         c0,         k0,        e_tot

###   1.00e+02,   2.00e+01,   8.00e+01,   -11.144174

{'kmesh': 32, 'method': 'fftisdf-jy', 'ncpu': 1, 'cell': 'diamond-prim', 'natm': 2, 't_build': 73.95, 't_vj': 1.35, 't_vk': 0.58, 'ke_cutoff': 100.0, 'c0': 20.0, 'k0': 80.0, 'e_tot': -11.144174}


AssertionError: 